# Inspecting TFX metadata


## Learning Objectives

1. Use a GRPC server to access and analyze pipeline artifacts stored in the ML Metadata service of your AI Platform Pipelines instance.

In this lab, you will explore TFX pipeline metadata including pipeline and run artifacts. A hosted **AI Platform Pipelines** instance includes the [ML Metadata](https://github.com/google/ml-metadata) service. In **AI Platform Pipelines**, ML Metadata uses *MySQL* as a database backend and can be accessed using a GRPC server.

## Setup

In [1]:
import os
import json

import ml_metadata
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma


from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

from tfx.orchestration import metadata
from tfx.types import standard_artifacts

from tensorflow.python.lib.io import file_io

In [2]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

TFX version: 0.25.0
KFP version: 1.0.4


### Option 1: Explore metadata from existing TFX pipeline runs from AI Pipelines instance created in `lab-02` or `lab-03`.

#### 1.1 Configure Kubernetes port forwarding

To enable access to the ML Metadata GRPC server, configure Kubernetes port forwarding.

From a JupyterLab terminal, execute the following commands:

```
gcloud container clusters get-credentials [YOUR CLUSTER] --zone [YOUR CLUSTER ZONE]  
kubectl port-forward  service/metadata-grpc-service --namespace [YOUR NAMESPACE] 7000:8080
```

Proceed to the next step, "Connecting to ML Metadata".

### Option 2: Create new AI Pipelines instance and evaluate metadata on newly triggered pipeline runs.

Hosted AI Pipelines incurs cost for the duration your Kubernetes cluster is running. If you deleted your previous lab instance, proceed with the 6 steps below to deploy a new TFX pipeline and triggers runs to inspect its metadata.

In [3]:
import yaml

# Set `PATH` to include the directory containing TFX CLI.
PATH=%env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


The pipeline source can be found in the `pipeline` folder. Switch to the `pipeline` folder and compile the pipeline.

In [4]:
%cd pipeline

/home/jupyter/mlops-on-gcp/workshops/tfx-caip-tf23/lab-04-tfx-metadata/solutions/pipeline


#### 2.1 Create AI Platform Pipelines cluster

Navigate to [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

Create or select an existing Kubernetes cluster (GKE) and deploy AI Platform. Make sure to select `"Allow access to the following Cloud APIs https://www.googleapis.com/auth/cloud-platform"` to allow for programmatic access to your pipeline by the Kubeflow SDK for the rest of the lab. Also, provide an `App instance name` such as "TFX-lab-04".

#### 2.2 Configure environment settings

Update  the below constants  with the settings reflecting your lab environment.

- `GCP_REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `kubeflowpipelines-` prefix. Alternatively, you can specify create a new storage bucket to write pipeline artifacts to. When specifying the bucket, do not use the trailing slash (/) at the end of the bucket name.

In [5]:
!gsutil ls

gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default/


* `CUSTOM_SERVICE_ACCOUNT` - In the gcp console Click on the Navigation Menu. Navigate to `IAM & Admin`, then to `Service Accounts` and use the service account starting with prefix - `'tfx-tuner-caip-service-account'`. This enables CloudTuner and the Google Cloud AI Platform extensions Tuner component to work together and allows for distributed and parallel tuning backed by AI Platform Vizier's hyperparameter search algorithm. Please see the lab setup `README` for setup instructions.

- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. The endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

In [6]:
#TODO: Set your environment resource settings here for GCP_REGION, ARTIFACT_STORE_URI, ENDPOINT, and CUSTOM_SERVICE_ACCOUNT.
GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default' #Change
ENDPOINT = '33448464655cef9-dot-us-central1.pipelines.googleusercontent.com' #Change
CUSTOM_SERVICE_ACCOUNT = 'tfx-tuner-caip-service-account@qwiklabs-gcp-04-fe01d95f44d5.iam.gserviceaccount.com' #Change

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [7]:
# Set your resource settings as environment variables. These override the default values in pipeline/config.py.
%env GCP_REGION={GCP_REGION}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env CUSTOM_SERVICE_ACCOUNT={CUSTOM_SERVICE_ACCOUNT}
%env PROJECT_ID={PROJECT_ID}

env: GCP_REGION=us-central1
env: ARTIFACT_STORE_URI=gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default
env: CUSTOM_SERVICE_ACCOUNT=tfx-tuner-caip-service-account@qwiklabs-gcp-04-fe01d95f44d5.iam.gserviceaccount.com
env: PROJECT_ID=qwiklabs-gcp-04-fe01d95f44d5


#### 2.3 Compile pipeline

In [8]:
PIPELINE_NAME = 'tfx_covertype_lab_04'
MODEL_NAME = 'tfx_covertype_classifier'
DATA_ROOT_URI = 'gs://cloud-training/OCBL203/workshop-datasets'
CUSTOM_TFX_IMAGE = 'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)
RUNTIME_VERSION = '2.3'
PYTHON_VERSION = '3.7'
USE_KFP_SA=False
ENABLE_TUNING=True

In [9]:
%env PIPELINE_NAME={PIPELINE_NAME}
%env MODEL_NAME={MODEL_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env KUBEFLOW_TFX_IMAGE={CUSTOM_TFX_IMAGE}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERIONS={PYTHON_VERSION}
%env USE_KFP_SA={USE_KFP_SA}
%env ENABLE_TUNING={ENABLE_TUNING}

env: PIPELINE_NAME=tfx_covertype_lab_04
env: MODEL_NAME=tfx_covertype_classifier
env: DATA_ROOT_URI=gs://cloud-training/OCBL203/workshop-datasets
env: KUBEFLOW_TFX_IMAGE=gcr.io/qwiklabs-gcp-04-fe01d95f44d5/tfx_covertype_lab_04
env: RUNTIME_VERSION=2.3
env: PYTHON_VERIONS=3.7
env: USE_KFP_SA=False
env: ENABLE_TUNING=True


In [10]:
!tfx pipeline compile --engine kubeflow --pipeline_path runner.py

CLI
Compiling pipeline
Pipeline compiled successfully.
Pipeline package path: /home/jupyter/mlops-on-gcp/workshops/tfx-caip-tf23/lab-04-tfx-metadata/solutions/pipeline/tfx_covertype_lab_04.tar.gz


#### 2.4 Deploy pipeline to AI Platform

In [11]:
!tfx pipeline create  \
--pipeline_path=runner.py \
--endpoint={ENDPOINT} \
--build_target_image={CUSTOM_TFX_IMAGE}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
[Skaffold] Generating tags...
[Skaffold]  - gcr.io/qwiklabs-gcp-04-fe01d95f44d5/tfx_covertype_lab_04 -> gcr.io/qwiklabs-gcp-04-fe01d95f44d5/tfx_covertype_lab_04:latest
[Skaffold] Checking cache...
[Skaffold]  - gcr.io/qwiklabs-gcp-04-fe01d95f44d5/tfx_covertype_lab_04: Not found. Building
[Skaffold] Starting build...
[Skaffold] Building [gcr.io/qwiklabs-gcp-04-fe01d95f44d5/tfx_covertype_lab_04]...
[Skaffold] Sending build context to Docker daemon   59.9kB
[Skaffold] Step 1/4 : FROM tensorflow/tfx:0.25.0
[Skaffold] 0.25.0: Pulling from tensorflow/tfx
[Skaffold] bd47987755ba: Pulling fs layer
[Skaffold] 831c222b21d8: Pulling fs layer
[Skaffold] 3c2cba919283: Pulling fs layer
[Skaffold] e378d88a5f59: Pulling fs layer
[Skaffold] df37508d2f5c: Pulling fs layer
[Skaffold] c28e7cc900d1: Pulling fs layer
[Skaffold] 9019978541a7: Pulling fs layer
[Skaffold]

(optional) If you make local changes to the pipeline, you can update the deployed package on AI Platform with the following command:

In [ ]:
!tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}

#### 2.5 Create and monitor pipeline run

In [12]:
!tfx run create --pipeline_name={PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: tfx_covertype_lab_04
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: tfx_covertype_lab_04
+----------------------+--------------------------------------+----------+---------------------------+----------------------------------------------------------------------------------------------------------------------------+
| pipeline_name        | run_id                               | status   | created_at                | link                                                                                                                       |
+======================+======================================+==========+===========================+============================================================================================================================+
| tfx_covertype_lab_04 | 972d72fb-a4e1-4734-8073-1f3000444746 | Pending  | 2024-05-17T06:12:25+00:00 | http://33448464655cef9-dot-u

#### 2.6 Configure Kubernetes port forwarding

To enable access to the ML Metadata GRPC server, configure Kubernetes port forwarding.

From a JupyterLab terminal, execute the following commands:

```
gcloud container clusters get-credentials [YOUR CLUSTER] --zone [YOURE CLUSTER ZONE]  
kubectl port-forward  service/metadata-grpc-service --namespace [YOUR NAMESPACE] 7000:8080
```

## Connecting to ML Metadata 

### Configure ML Metadata GRPC client

In [16]:
grpc_host = 'localhost'
grpc_port = 7000
connection_config = metadata_store_pb2.MetadataStoreClientConfig()
connection_config.host = grpc_host
connection_config.port = grpc_port

### Connect to ML Metadata service

In [17]:
store = metadata_store.MetadataStore(connection_config)

### Important

A full pipeline run without tuning takes about 40-45 minutes to complete. You need to wait until a pipeline run is complete before proceeding with the steps below.

## Exploring ML Metadata 

The Metadata Store uses the following data model:

- `ArtifactType` describes an artifact's type and its properties that are stored in the Metadata Store. These types can be registered on-the-fly with the Metadata Store in code, or they can be loaded in the store from a serialized format. Once a type is registered, its definition is available throughout the lifetime of the store.
- `Artifact` describes a specific instances of an ArtifactType, and its properties that are written to the Metadata Store.
- `ExecutionType` describes a type of component or step in a workflow, and its runtime parameters.
- `Execution` is a record of a component run or a step in an ML workflow and the runtime parameters. An Execution can be thought of as an instance of an ExecutionType. Every time a developer runs an ML pipeline or step, executions are recorded for each step.
- `Event` is a record of the relationship between an Artifact and Executions. When an Execution happens, Events record every Artifact that was used by the Execution, and every Artifact that was produced. These records allow for provenance tracking throughout a workflow. By looking at all Events MLMD knows what Executions happened, what Artifacts were created as a result, and can recurse back from any Artifact to all of its upstream inputs.
- `ContextType` describes a type of conceptual group of Artifacts and Executions in a workflow, and its structural properties. For example: projects, pipeline runs, experiments, owners.
- `Context` is an instances of a ContextType. It captures the shared information within the group. For example: project name, changelist commit id, experiment annotations. It has a user-defined unique name within its ContextType.
- `Attribution` is a record of the relationship between Artifacts and Contexts.
- `Association` is a record of the relationship between Executions and Contexts.

List the registered artifact types.

In [18]:
for artifact_type in store.get_artifact_types():
    print(artifact_type.name)

Schema
Examples
ExampleStatistics
TransformGraph
TransformCache
ExampleAnomalies
HyperParameters


Display the registered execution types.

In [19]:
for execution_type in store.get_execution_types():
    print(execution_type.name)

DummyExecutionType
tfx.components.common_nodes.importer_node.ImporterNode
tfx.components.common_nodes.resolver_node.ResolverNode
tfx.components.example_gen.csv_example_gen.component.CsvExampleGen
tfx.components.transform.component.Transform
tfx.components.statistics_gen.component.StatisticsGen
tfx.components.example_validator.component.ExampleValidator
tfx.components.schema_gen.component.SchemaGen
tfx.extensions.google_cloud_ai_platform.tuner.component.Tuner


List the registered context types.

In [20]:
for context_type in store.get_context_types():
    print(context_type.name)

pipeline
run
component_run


## Visualizing TFX artifacts

### Retrieve data analysis and validation artifacts

In [21]:
with metadata.Metadata(connection_config) as store:
    schema_artifacts = store.get_artifacts_by_type(standard_artifacts.Schema.TYPE_NAME)    
    stats_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleStatistics.TYPE_NAME)
    anomalies_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleAnomalies.TYPE_NAME)

In [22]:
schema_file = os.path.join(schema_artifacts[-1].uri, 'schema.pbtxt')
print("Generated schame file:{}".format(schema_file))

stats_path = stats_artifacts[-1].uri
train_stats_file = os.path.join(stats_path, 'train', 'stats_tfrecord')
eval_stats_file = os.path.join(stats_path, 'eval', 'stats_tfrecord')
print("Train stats file:{}, Eval stats file:{}".format(
    train_stats_file, eval_stats_file))

anomalies_path = anomalies_artifacts[-1].uri
train_anomalies_file = os.path.join(anomalies_path, 'train', 'anomalies.pbtxt')
eval_anomalies_file = os.path.join(anomalies_path, 'eval', 'anomalies.pbtxt')

print("Train anomalies file:{}, Eval anomalies file:{}".format(
    train_anomalies_file, eval_anomalies_file))

Generated schame file:gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default/tfx_covertype_lab_04/972d72fb-a4e1-4734-8073-1f3000444746/SchemaGen/schema/7/schema.pbtxt
Train stats file:gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default/tfx_covertype_lab_04/972d72fb-a4e1-4734-8073-1f3000444746/StatisticsGen/statistics/5/train/stats_tfrecord, Eval stats file:gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default/tfx_covertype_lab_04/972d72fb-a4e1-4734-8073-1f3000444746/StatisticsGen/statistics/5/eval/stats_tfrecord
Train anomalies file:gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default/tfx_covertype_lab_04/972d72fb-a4e1-4734-8073-1f3000444746/ExampleValidator/anomalies/6/train/anomalies.pbtxt, Eval anomalies file:gs://qwiklabs-gcp-04-fe01d95f44d5-kubeflowpipelines-default/tfx_covertype_lab_04/972d72fb-a4e1-4734-8073-1f3000444746/ExampleValidator/anomalies/6/eval/anomalies.pbtxt


### Visualize schema

In [23]:
schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Visualize statistics

#### Exercise: looking at the features visualized below, answer the following questions:

- Which feature transformations would you apply to each feature with TF Transform?
- Are there data quality issues with certain features that may impact your model performance? How might you deal with it?

In [24]:
train_stats = tfdv.load_statistics(train_stats_file)
eval_stats = tfdv.load_statistics(eval_stats_file)
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### Visualize anomalies

In [25]:
train_anomalies = tfdv.load_anomalies_text(train_anomalies_file)
tfdv.display_anomalies(train_anomalies)

In [26]:
eval_anomalies = tfdv.load_anomalies_text(eval_anomalies_file)
tfdv.display_anomalies(eval_anomalies)

### Retrieve model artifacts

In [ ]:
with metadata.Metadata(connection_config) as store:
    model_eval_artifacts = store.get_artifacts_by_type(standard_artifacts.ModelEvaluation.TYPE_NAME)
    hyperparam_artifacts = store.get_artifacts_by_type(standard_artifacts.HyperParameters.TYPE_NAME)
    
model_eval_path = model_eval_artifacts[-1].uri
print("Generated model evaluation result:{}".format(model_eval_path))
best_hparams_path = os.path.join(hyperparam_artifacts[-1].uri, 'best_hyperparameters.txt')
print("Generated model best hyperparameters result:{}".format(best_hparams_path))

### Return best hyperparameters

In [ ]:
# Latest pipeline run Tuner search space.
json.loads(file_io.read_file_to_string(best_hparams_path))['space']

In [ ]:
# Latest pipeline run Tuner searched best_hyperparameters artifacts.
json.loads(file_io.read_file_to_string(best_hparams_path))['values']

### Visualize model evaluations

#### Exercise: review the model evaluation results below and answer the following questions:

- Which Wilderness Area had the highest accuracy?
- Which Wilderness Area had the lowest performance? Why do you think that is? What are some steps you could take to improve your next model runs?

In [ ]:
eval_result = tfma.load_eval_result(model_eval_path)
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Wilderness_Area')

**Debugging tip**: If the TFMA visualization of the Evaluator results do not render, try switching to view in a Classic Jupyter Notebook. You do so by clicking `Help > Launch Classic Notebook` and re-opening the notebook and running the above cell to see the interactive TFMA results.

## License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>
